In [1]:
import pandas as pd
import wikipedia
import re
import json
import random
from nltk import ngrams
from IPython.core.debugger import set_trace

random.seed(1234)
pd.set_option('display.max_colwidth', -1)

In [2]:
train_df = pd.read_json('/data/ai_challenge/vietnameseqa/data/train.json', encoding= 'utf-8')
train_df.head()

,id,label,question,text,title
0,u7-1570446247_1,True,Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi,"Năm 2013 , Nguyễn Quang Hải giành chức vô địch U21 quốc gia 2013 cùng với đội trẻ Hà Nội T&T và tạo nên cú sốc khi trở thành cầu thủ 16 tuổi đầu tiên giành được danh hiệu vô địch U21 quốc gia .",Nguyễn Quang Hải (sinh 1997)
1,u7-1570446247_2,True,Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi,"Sau chức vô địch U-21 quốc gia 2013 , Nguyễn Quang Hải mới 16 tuổi lập tức được HLV Phan Thanh Hùng điền vào danh sách của đội bóng thủ đô tham dự V-League 2014 .",Nguyễn Quang Hải (sinh 1997)
2,u7-1570446247_0,False,Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi,Anh bắt đầu gia nhập lò đào tạo trẻ Hà Nội T&T khi mới 9 tuổi vào năm 2006 .,Nguyễn Quang Hải (sinh 1997)
3,u7-1570446247_3,False,Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi,"Năm 2011 , Nguyễn Quang Hải mới 14 tuổi được gọi lên đội tuyển U-16 Việt Nam , thi đấu với những cầu thủ lớn tuổi hơn tại vòng loại giải vô địch bóng đá U-16 châu Á.",Nguyễn Quang Hải (sinh 1997)
4,u7-1570445661_0,True,Mỗi hiệp bóng đá kéo dài bao lâu,"Một trận đấu bóng đá thông thường có hai hiệp , mỗi hiệp 45 phút với khoảng thời gian 15 phút nghỉ giữa hai hiệp .",Bóng đá


In [3]:
data_dict = {}
for index, row in train_df.iterrows():
    id = row['id']
    label = row['label']
    question = row['question']
    text = row['text']
    title = row['title']
    
    if title not in data_dict:
        data_dict[title] = []
    data_dict[title].append((id, label, question, text)) 

In [4]:
def export_grams(s, n):
    n_grams = ngrams(s.split(), n)
    n_grams_set = set()
    for grams in n_grams:
        n_grams_set.add(grams)
    
    return n_grams_set

def export_n_grams(s):
    return set.union(export_grams(s, 2), export_grams(s, 3))

## Read Wiki Data

In [5]:
wiki_dict = {}
wiki_data = ''
with open('/data/ai_challenge/vietnameseqa/data/wiki_2.txt', 'r') as file:
    for line in file:
        wiki_data += line

for content in wiki_data.split('}end}\n'):
    content_arr = content.split('} : {{')
    
    title = content_arr[0]
    title = title[1: len(title)]
    try:
        para = content_arr[1]
    except: 
        print(content)
        continue
    wiki_dict[title] = para

## Export Data

In [6]:
id_count = 0
for key, value in data_dict.items():
    print("***Process {}***".format(key))
    if key not in wiki_dict:
        continue
    wiki_page = wiki_dict[key]
        
    paragraphs = wiki_page.split('\n')
    para_list = []
    for para in paragraphs:
        if '==' not in para and para != '' and len(para.split())>=15:
            para = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', para)
            para_list.append(para.strip())
            
    if len(para_list) == 0:
        continue
    match_index = []
    for item in value:
        (id, label, question, text) = item
        text_sent = text.split('.')
        if '' in text_sent:
            text_sent.remove('')
        match_para_index = None
        
        text_grams = export_n_grams(text)
        max_inter = 0
        for para_index, para in enumerate(para_list):
            para_grams = export_n_grams(para)
            intersec = len(set.intersection(text_grams, para_grams))
            if max_inter <= intersec:
                max_inter = intersec
                match_para_index = para_index
#         for para_index, para in enumerate(para_list):
#             para_sents = para.split('.')
#             for sent in text_sent: 
#                 for para_sent in para_sents:
#                     if re.sub(r'[\W_]+', '', sent) in re.sub(r'[\W_]+', '', para_sent):
#                         match = True
#                         match_para_index = para_index
#                         break
                        
#                 if match_para_index: 
#                     break
#             if match_para_index: 
#                 break
    
        para_sents = para_list[match_para_index].split('.') 
        match_index.append(match_para_index)
        if label:
            set_min = False
            min_index = 0
            max_index = 0
            first_sent = text_sent[0]
            last_sent = text_sent[-1]
            first_max_inter = 0
            last_max_inter = 0
            
            for index, para_sent in enumerate(para_sents):
                first_inter = len(set.intersection(export_n_grams(first_sent), export_n_grams(para_sent)))
                if first_max_inter < first_inter:
                    first_max_inter = first_inter
                    min_index = index
                
                last_inter = len(set.intersection(export_n_grams(last_sent), export_n_grams(para_sent)))
                if last_max_inter < last_inter:
                    last_max_inter = last_inter
                    max_index = index

            if min_index <= max_index:
                if min_index > 0:
                    min_index = min_index - 1

                new_text = '.'.join(para_sents[min_index : max_index + 2])
            else:
                new_text = '.'.join(para_sents)

            # consider here for fine-tune
            id_count += 1
            train_df = train_df.append({'id': 'add_' + str(id_count), 'label': label, 
                                       'question': question, 'text': new_text, 'title': key}, ignore_index=True)

    # Add random false case
#     (id, label, question, text) = value[0]
#     non_match_para = []
#     for index, para in enumerate(para_list):
#         if index not in match_index:
#             non_match_para.append(para)
    
#     if len(non_match_para) > 2:
#         random_indices = random.sample(range(0, len(non_match_para)), 2)
#     elif len(non_match_para) == 2:
#         random_indices = [0, 1]
#     elif len(non_match_para) == 1:
#         random_indices = [0]
#     else:
#         continue
        
#     for random_index in random_indices:
#         random_para = non_match_para[random_index]
#         para_sents = random_para.split('.')

#         id_count += 1
#         new_text = '.'.join(para_sents[:random.randrange(1, 15, 1)])
#         train_df = train_df.append({'id': 'add_' + str(id_count), 'label': False, 
#                                        'question': question, 'text': new_text, 'title': key}, ignore_index=True)

***Process Nguyễn Quang Hải (sinh 1997)***
***Process Bóng đá***
***Process Quân đội Hoa Kỳ***
***Process Lực lượng Dã chiến I , Việt Nam***
***Process NATO***
***Process Ngọc Lan***
***Process Vương Thị Ngọc Lan***
***Process Danh sách giải thưởng và thành tựu của Thu Phương***
***Process Trần Lê Quỳnh***
***Process Na Uy***
***Process Liên minh giữa Thuỵ Điển và Na Uy***
***Process Greenland***
***Process Xuân Quỳnh***
***Process Lưu Minh Vũ***
***Process Đế quốc Mông Cổ***
***Process Kim loại***
***Process Liên kết kim loại***
***Process Kim loại yếu***
***Process Bangladesh***
***Process Đội tuyển Davis Cup Bangladesh***
***Process Zlatan Ibrahimović***
***Process Hy Lạp***
***Process Thế vận hội dành cho người khuyết tật***
***Process ***
***Process Địa lý Ấn Độ***
***Process Sundarban***
***Process Bảng Anh***
***Process Bảng xếp hạng đại học thế giới Quacquarelli Symonds***
***Process Giải vô địch bóng đá thế giới 1966 ( Bảng 1 )***
***Process Tôn Trung Sơn***
***Process Hoàng H

***Process Lịch sử hành chính Nam Định***
***Process Blockchain***
***Process Điện Biên***
***Process Fate / stay night***
***Process Chùa Âng***
***Process Vịnh Xuân quyền***
***Process Vịnh xuân quyền ( phim truyền hình )***
***Process Võ Hoàng Yên***
***Process Khmer Đỏ***
***Process Đảng Nhân dân Cách mạng Khmer***
***Process Đảng Nhân dân Campuchia***
***Process Quan Âm***
***Process Sa Tăng***
***Process Lư Thắng Ngạn***
***Process Bồ Tát***
***Process Đại hội Thể thao châu Á 2018***
***Process Bóng đá tại Đại hội Thể thao châu Á 2018***
***Process Wushu tại Đại hội Thể thao châu Á 2018 - Trường quyền nam***
***Process Axit axetic***
***Process Giấm táo***
***Process Kiểm soát loài gây hại***
***Process Ẩm thực Nhật Bản***
***Process Biển Na Uy***
***Process Haakon , Thái tử của Na Uy***
***Process Rissa , Na Uy***
***Process Vật lý thiên văn***
***Process Vật lý nguyên tử***
***Process Vật lý lý thuyết***
***Process Crom***
***Process Crôm trioxit***
***Process Crôm ( III ) pico

***Process Phần mềm***
***Process Hệ thống điều khiển***
***Process Thư viện phần mềm khoa học GNU***
***Process Pascal ( đơn vị )***
***Process Inch nước***
***Process Đơn vị đo công suất***
***Process Kilômét***
***Process George Washington***
***Process Thống tướng lục quân ( Hoa Kỳ )***
***Process Nhảy xa ba bước***
***Process Aveiro ( tỉnh )***
***Process A-dục vương***
***Process Võ thuật trong tiểu thuyết Kim Dung***
***Process Đế quốc Nhật Bản***
***Process Phương diện quân ( Đế quốc Nhật Bản )***
***Process Tập đoàn quân 11 ( Đế quốc Nhật Bản )***
***Process Thiên hoàng Taishō***
***Process Áp suất***
***Process Đo áp suất***
***Process Thuỷ lực cắt phá***
***Process Áp suất chất lỏng***
***Process Bojan Krkić***
***Process Nguyễn Văn Chi***
***Process Nguyễn Xuân Anh***
***Process Biên Hoà***
***Process Đồng Nai***
***Process Hổ***
***Process Machali***
***Process Beechcraft King Air***
***Process Tên ký hiệu của NATO***
***Process George Washington ( định hướng )***
***Proce

***Process Đô thị Việt Nam***
***Process Nhà máy thuỷ điện Hoà Bình***
***Process Phó tổng thống Brasil***
***Process Hạt hạ nguyên tử***
***Process Tự nhiên***
***Process Ngày Trái Đất***
***Process Trung du và miền núi phía Bắc***
***Process Nhà máy lọc dầu Dung Quất***
***Process Nhà máy điện hạt nhân Obninsk***
***Process George I của Liên hiệp Anh***
***Process Titanic ( phim 1997 )***
***Process Jacquou , người nông dân nổi dậy ( phim )***
***Process Minh Hải***
***Process Sông Ông Đốc ( quận )***
***Process Liên Xô***
***Process Sự sụp đổ của Liên Xô và Đông Âu***
***Process Cộng hoà Xã hội chủ nghĩa Xô viết Liên bang Nga***
***Process Phân cấp hành chính Nga***
***Process Vlaamse Ruit***
***Process Đội tuyển bóng đá quốc gia Cộng hoà Ireland***
***Process Lê Hồng Anh***
***Process Đại hội Đảng Cộng sản Việt Nam XII***
***Process Tỉnh uỷ Hà Nam***
***Process Facebook***
***Process Facebook Messenger***
***Process Lịch sử châu Mỹ***
***Process Phạm Bình Minh***
***Process Hiến ph

***Process Nhiệm kỳ tổng thống Donald Trump***
***Process Trường Sa , Khánh Hoà***
***Process Xung đột giành quyền kiểm soát tại quần đảo Trường Sa 1988***
***Process Quần đảo Trường Sa***
***Process Tây Ban Nha***
***Process România***
***Process Andorra***
***Process Pháp***
***Process EXID***
***Process UJi ( ca sĩ )***
***Process Shinsadong Tiger***
***Process Tố Hữu***
***Process Anime***
***Process Loveless ( anime )***
***Process Trương Tấn Sang***
***Process Nguyễn Văn Lập***
***Process Đại hội Đảng Cộng sản Việt Nam XI***
***Process Ngũ hổ tướng ( Tam Quốc diễn nghĩa )***
***Process Tam quốc diễn nghĩa ( trò chơi điện tử )***
***Process Hán Quang Vũ Đế***
***Process Bành Sủng***
***Process Trận Côn Dương***
***Process Nhà Lý***
***Process Chiến tranh Nguyên Mông – Đại Việt***
***Process Phố cổ Hội An***
***Process Khối lượng riêng***
***Process Năng lượng riêng***
***Process Lỗ đen siêu khối lượng***
***Process Khối lượng không khí***
***Process Chế độ chính trị***
***Process 

...

***Process Doãn phu nhân (Tống Thái Tông)***
***Process Chuyến bay 370 của Malaysia Airlines***
***Process Sách trắng quốc phòng (Việt Nam)***
***Process Chủ tịch Đại Hội đồng Liên Hiệp Quốc***
***Process Quốc hội Liên bang Đức***
***Process Hòn Mun***
***Process Jylland***
***Process Tiến hóa loài người***
***Process Cổ sinh vật học***
***Process Thịt chế biến sẵn***
***Process Vườn quốc gia Phước Bình***
***Process Hệ động vật***
***Process Khu du lịch sinh thái Gáo Giồng***
***Process Cá biển***
***Process Cá mòi dầu Ấn Độ***
***Process Danh sách các loài cá nước ngọt ở Đồng bằng sông Cửu Long***
***Process Nguyễn Hữu An***
***Process Nguyễn Quang Bích (thiếu tướng)***
***Process Hoàng Cầm (tướng)***
***Process Khu di tích chiến trường Điện Biên Phủ***
***Process Nhà hát Chèo Hưng Yên***
***Process Thẻ điện thoại***
***Process Nguyễn Nhiêu Cốc***
***Process Tiếng Tày***
***Process Tương tác thiên hà***
***Process Mark Tuan***
***Process Nghệ thuật***
***Process Trường phái trừu tượn

***Process Bát đẳng Chơn Hồn***
***Process Thế Eocen***
***Process Hòa Hải, Ngũ Hành Sơn***
***Process Vườn quốc gia Pirin***
***Process Seibersdorf***
***Process Đường vành đai 4 (Hà Nội)***
***Process Súng tiểu liên***
***Process AK-107***
***Process GP-25***
***Process Súng***
***Process Tổ chức Lương thực và Nông nghiệp Liên Hiệp Quốc***
***Process Ngày Lương thực thế giới***
***Process Alfonso García Robles***
***Process Hiệp định về Vùng nước lịch sử Việt Nam - Campuchia***
***Process Hiệp định về phân định ranh giới thềm lục địa Việt Nam-Indonesia***
***Process Hiệp định hợp tác nghề cá ở Vịnh Bắc Bộ***
***Process Cố Mạn***
***Process Jane Austen***
***Process Lý Hải***
***Process Richard Matheson***
***Process Karakoram***
***Process George Mallory***
***Process Alison Hargreaves***
***Process Hòa giải và hòa hợp dân tộc ở Việt Nam***
***Process Ghế điện***
***Process Charles Francis Jenkins***
***Process Leon Theremin***
***Process Sau sau***
***Process Hồng Trung Hoa***
***Pr

***Process Trương Thị Bích Hạnh***
***Process Trần Thanh Liêm***
***Process Cảnh sát biển Việt Nam***
***Process Quân đội Bangladesh***
***Process CSB-8005***
***Process Lê Hồng Thủy Tiên***
***Process Kha Vạng Cân***
***Process Danh sách các cuộc viếng thăm Việt Nam của thủ tướng Hungary***
***Process Ngọc Trạo, thành phố Thanh Hóa***
***Process Mỹ Quới***
***Process Giai Phạm***
***Process Nguyễn Văn Linh (Bắc Giang)***
***Process Salem, Oregon***
***Process Chantrea***
***Process Kampong Trabaek***
***Process Trần Quốc Trung (chính trị gia)***
***Process Quận 1, Lyon***
***Process Trần Kim Yến***
***Process Hoyt Lakes, Minnesota***
***Process Quận Gwangsan***
***Process Đường tỉnh 237***
***Process Nong Het***
***Process Mukdahan (tỉnh)***
***Process Lai Châu***
***Process Tổng cục Du lịch (Việt Nam)***
***Process Tỉnh ủy Thừa Thiên - Huế***
***Process Hồ Ngọc Lân***
***Process Ủy ban kỹ thuật điện quốc tế***
***Process Bộ Công an Cộng hòa Nhân dân Trung Hoa***
***Process Bộ trưởng 

***Process Panama***
***Process Tài nguyên nước***
***Process Giao thông Thành phố Hồ Chí Minh***
***Process Sở Cảnh sát thành phố Los Angeles***
***Process Nhà nước cảnh sát***
***Process Ủy ban Liên chính phủ về Biến đổi Khí hậu***
***Process Nghị định thư Kyōto***
***Process Tỷ lệ lạm phát***
***Process Đình lạm***
***Process Danh sách quốc gia theo tỉ lệ lạm phát***
***Process Diễn đàn Khu vực ASEAN***
***Process Vũ Đăng Toàn***
***Process Françoise Demulder***
***Process T-54/55***
***Process Bảo tàng Lịch sử Quân sự Việt Nam***
***Process Chiến tranh Afghanistan (1978–1992)***
***Process Phan Đình Giót***
***Process Cẩm Xuyên***
***Process Trừ Văn Thố***
***Process Lộ Bác Đức***
***Process Lực lượng Thanh niên xung phong thành phố Hồ Chí Minh***
***Process Modem DSL***
***Process Slirp***
***Process Samsung Kies***
***Process Điện tử tiêu dùng***
***Process Hệ thống điện khí hóa đường sắt***
***Process Ngọc Hà, Ba Đình***
***Process Nam có quan hệ tình dục với nam***
***Process C

***Process Đồng tính luyến ái***
***Process Kết hôn giả***
***Process Xung đột Ả Rập-Israel***
***Process Quan hệ Israel – Thổ Nhĩ Kỳ***
***Process Tâm linh***
***Process Linh Linh***
***Process Linh Tâm***
***Process Trang Thanh Xuân***
***Process Linh hồn***
***Process Nguyễn Hữu Thọ***
***Process Hoàng Sâm***
***Process Mười lời thề danh dự của quân nhân Quân đội nhân dân Việt Nam***
***Process Peter Henlein***
***Process Đồng hồ báo thức***
***Process Christiaan Huygens***
***Process Sierra Leone***
***Process Năng lượng ở Cộng hòa Dân chủ Congo***
***Process Kindu***
***Process Zaire (tỉnh)***
***Process Nguyễn Đức Minh***
***Process Bộ Tư pháp (Ai Cập)***
***Process Bùi Thiện Ngộ***
***Process Nguyễn Phi Hùng (thiếu tướng)***
***Process Mai Trung Thứ***
***Process Garçon à la pipe***
***Process Margaret Keane***
***Process Dafne Keen***
***Process Nguyễn Cao Kỳ Duyên***
***Process Nguyễn Cao Kỳ Duyên (hoa hậu)***
***Process Kỳ Duyên***
***Process Trần Tiểu Vy***
***Process Thượng

In [7]:
train_df.tail(50)

,id,label,question,text,title
22574,add_4467,True,Con vật đầu tiên được nhân bản vô tính thành công,"Một con cừu mouflon đã nhân bản thành công vào đầu năm 2001 và đã sống ít nhất bảy tháng , khiến nó trở thành bản sao đầu tiên của một động vật có vú bị đe dọa để tồn tại . Sau hàng loạt thất bại trong việc nhân bản vô tính thú hoang , các nhà khoa học châu Âu đã công bố thành tựu về một con cừu mouflon quý hiếm được nhân bản đã trải qua 6 tháng tuổi mà không gặp sự cố nào . Con cừu cái này đang được chăm sóc tại Trung tâm Cứu hộ động vật hoang dã ở Sardinia , Italia . Để tạo ra con cừu mouflon này , nhóm khoa học của Đại học Teramo đã áp dụng kỹ thuật tương tự như kỹ thuật đã thành công trên cừu Dolly . ADN được trích ra từ xác của một con cừu cái đã chết trong Trung tâm Cứu hộ động vật hoang dã ở Sardinia .",Mouflon
22575,add_4468,True,Con vật đầu tiên được nhân bản vô tính thành công,"Một con cừu mouflon đã nhân bản thành công vào đầu năm 2001 và đã sống ít nhất bảy tháng , khiến nó trở thành bản sao đầu tiên của một động vật có vú bị đe dọa để tồn tại . Sau hàng loạt thất bại trong việc nhân bản vô tính thú hoang , các nhà khoa học châu Âu đã công bố thành tựu về một con cừu mouflon quý hiếm được nhân bản đã trải qua 6 tháng tuổi mà không gặp sự cố nào . Con cừu cái này đang được chăm sóc tại Trung tâm Cứu hộ động vật hoang dã ở Sardinia , Italia . Để tạo ra con cừu mouflon này , nhóm khoa học của Đại học Teramo đã áp dụng kỹ thuật tương tự như kỹ thuật đã thành công trên cừu Dolly . ADN được trích ra từ xác của một con cừu cái đã chết trong Trung tâm Cứu hộ động vật hoang dã ở Sardinia .",Mouflon
22576,add_4469,True,Con vật đầu tiên được nhân bản vô tính thành công,"Cừu Dolly ( hay còn gọi là cừu nhân bản ) ( 5 tháng 7 năm 1996 - 14 tháng 2 năm 2003 ) là động vật có vú đầu tiên được nhân bản vô tính trên thế giới . Nó được tạo ra bởi Ian Wilmut , Keith Campbell và các cộng sự tại Viện Roslin ở Edinburgh , Scotland . Nó là con cừu thuộc giống cừu Dorset Phần Lan .",Cừu Dolly
22577,add_4470,True,Con vật đầu tiên được nhân bản vô tính thành công,"Sinh sản vô tính được chú ý nhiều và thường xuyên được nhắc đến trên các phương tiện thông tin đại chúng khi cừu Dolly - lần đầu tiên một động vật có vú được nhân bản . Sau đó hàng loạt các động vật khác được nhân bản như chó , lợn , dê",Dòng hóa
22578,add_4471,True,Bộ trưởng Bộ giáo dục Việt Nam là ai,"Năm 2014 , GS trẻ nhất và GS cao tuổi nhất đợt này cũng là GS trẻ nhất và cao tuổi nhất trong lịch sử ( 1974 - 2014 ) . Đó là GS . TS Phan Thanh Sơn Nam , 37 tuổi , chuyên ngành Hóa học , giảng viên trường ĐH Bách khoa – ĐHQGH TP . HCM và GS Lê Ngọc Canh , 81 tuổi , chuyên ngành Nghệ thuật , nguyên giảng viên trường ĐH Sư phạm Nghệ thuật TƯ – HN . Đội ngũ giáo sư , phó giáo sư được phong nhiều nhất ở các ngành: Y , Kinh tế , Toán , Lý , Hóa , Nông nghiệp . . . Năm 2018 , Hội đồng chức danh giáo sư Nhà nước công bố đầu tháng 2-2018 , 1 . 226 người được cho là đạt chuẩn giáo sư , phó giáo sư năm 2017 . Do dư luận phản ứng , Thủ tướng đã chỉ đạo Hội đồng chức danh rà soát lại danh sách này . Bộ trưởng GD-ĐT Phùng Xuân Nhạ - chủ tịch Hội đồng Chức danh giáo sư nhà nước , đã giao Thanh tra Bộ chủ trì xác minh làm rõ . Ngày 6-3 , trong danh sách mới 74 người được công nhận đạt tiêu chuẩn giáo sư , 1",Giáo sư (Việt Nam)
22579,add_4472,True,Tác giả của cuốn sách nổi tiếng Harry Potter,"Bộ tiểu thuyết Harry Potter của J . K . Rowling đã trở thành những tác phẩm được đọc nhiều nhất trong lịch sử văn học thiếu nhi , với độc giả từ mọi lứa tuổi và quốc gia . Vào tháng 5 năm 2008 , doanh số trên toàn thế giới của Harry Potter được thống kê lên đến trên 400 triệu bản , và những cuốn sách cũng được dịch từ nguyên bản tiếng Anh ra 67 thứ tiếng .",Harry Potter trong các phiên bản ngôn ngữ
22580,add_4473,True,đặc sản của Tuyên Quang là gì,"Khẩu Phái là một giống lúa nếp đặc sản của tỉnh Tuyên Quang , ở miền Bắc của Việt Nam . Lúa được trồng trong vụ Mùa . Hiện nay , đây là giống lúa được sản xuất theo 

## Add External Data

In [3]:
with open('/data/ai_challenge/vietnameseqa/data/external_data/train-v2.0.json') as json_file:
    data = json.load(json_file)['data']
    for item in data:
        paragraphs = item['paragraphs']
        title = item['title']
        
        for para in paragraphs:
            context = para['context']
            qas = para['qas']
            
            for quest_answer in qas:
                quest_id = quest_answer['id']
                question = quest_answer['question']
                is_possible = str(not quest_answer['is_impossible'])
                
                train_df = train_df.append({'id': quest_id, 'label': is_possible, 
                                       'question': question, 'text': context, 'title': title}, ignore_index=True)

## Export to File

In [8]:
output_arr = []
for index, row in train_df.iterrows():    
    output_arr.append(row.to_dict())
    
with open('/data/ai_challenge/vietnameseqa/data/train_update_true_only.json', 'w', encoding='utf8') as json_file:
    json.dump(output_arr, json_file, indent=4, ensure_ascii=False)

In [10]:
merge_df = pd.read_json('/data/ai_challenge/vietnameseqa/data/train_update_1.json', encoding= 'utf-8')

In [13]:
merge_df.tail(20)

,id,label,question,text,title
40078,add_21971,False,ai là người đầu tiên chế tạo ra bút bi,Có hai loại bút bi chính: loại dùng một lần và loại có thể nạp lại mực .,Bút Không Gian
40079,add_21972,False,ai là người đầu tiên chế tạo ra bút bi,"Khái niệm sử dụng một hòn bi trong một dụng cụ viết như một phương pháp áp dụng mực in vào giấy đã tồn tại từ cuối thế kỷ XIX . Trong những phát minh này , mực được đặt trong một ống mỏng , ở đầu bị chặn bởi một bi nhỏ , được đặt để nó không thể trượt vào trong ống hoặc rơi ra khỏi bút .",Bút Không Gian
40080,add_21973,True,tỉnh nào đông dân nhất Trung Quốc,"Dù con số thống kê chính thức thường cho rằng Quảng Đông xếp thứ 4 Trung Quốc về dân số với 80 triệu dân , các thông tin được công bố gần đây [1] cho rằng , có khoảng 30 triệu người nhập cư sống ở Quảng Đông ít nhất là 6 tháng mỗi năm , do đó dân số tỉnh này phải là 110 triệu người , là tỉnh đông dân nhất Trung Quốc . Làn sóng di cư của dân từ tỉnh khác đến tỉnh này , thường được gọi là ""dân trôi nổi"" là do sự phát triển kinh tế nhanh của Quảng Đông do đó cần nhiều lao động . Quảng Đông chính thức trở thành tỉnh đông dân nhất Trung Quốc từ tháng 1/2005 . Do vị trí địa lý giáp Thái Bình Dương , Quảng Đông cũng là quê cha đất tổ của một số lượng lớn Hoa Kiều",Quảng Đông
40081,add_21974,True,tỉnh nào đông dân nhất Trung Quốc,"Quảng Đông ( tiếng Trung: 广东; bính âm: Guǎngdōng ) là một tỉnh nằm ven bờ Biển Đông của Cộng hòa Nhân dân Trung Hoa . Quảng Đông là tỉnh đông nhất về số dân , đứng đầu về kinh tế Trung Quốc với 113 triệu dân và 9 , 73 nghìn tỷ NDT ( 1 , 47 nghìn tỷ USD ) năm 2018 . Tỉnh lị của Quảng Đông là Quảng Châu , đại đô thị này cùng với trung tâm kinh tế Thâm Quyến nằm trong số các thành phố đông dân và quan trọng nhất tại Trung Quốc . Vùng châu thổ Châu Giang là khu vực siêu đô thị lớn nhất thế giới . Từ năm 1989 , Quảng Đông đã đứng đầu về Tổng sản phẩm nội địa ( GDP ) so với các đơn vị cấp tỉnh khác , xếp sau đó lần lượt là Giang Tô và Sơn Đông",Quảng Đông
40082,add_21975,False,tỉnh nào đông dân nhất Trung Quốc,"Cùng với Quảng Tây , Quảng Đông được tạo lập thành một bộ phận của Lĩnh Nam đạo ( 嶺南道 ) , năm 627 vào thời Nhà Đường . Phần Quảng Đông trong đạo Lĩnh Nam được đổi tên thành Quảng Nam Đông Đạo năm 971 trong thời Nhà Tống - đây là sự bắt nguồn của cái tên Quảng Đông .",Quảng Đông
40083,add_21976,False,tỉnh nào đông dân nhất Trung Quốc,Bảng dưới đây cho thấy GDP của tỉnh Quảng Đông qua các năm với đơn vị tính là tỷ nhân dân tệ .,Quảng Đông
40084,add_21977,False,tỉnh nào đông dân nhất Trung Quốc,"An Đông , ( tiếng Trung: 安東省; bính âm: Āndōng shěng ) , là một tỉnh cũ tại Đông Bắc Trung Quốc , lãnh thổ của An Đông nay trở thành các phần của hai tỉnh Liêu Ninh và Cát Lâm . Tỉnh giáp với Triều Tiên ở phía đông nam qua sông Áp Lục . Tên của tỉnh có nguồn gốc từ An Đông đô hộ phủ dưới thời nhà Đường . Diện tích của tỉnh là 62 . 279 , 23 km² , dân số đầu năm 1947 ước tính vào khoảng 3 . 163 . 911 người , còn theo điều tra sau đó nửa năm là khoảng 2 . 971 . 170 người",An Đông (tỉnh)
40085,add_21978,False,tỉnh nào đông dân nhất Trung Quốc,"Tỉnh lị của An Đông trong thời kỳ 1934-1939 là Thông Hóa ( tiếng Trung: 通化; Wade-Giles: T'unghua ) . Tuy nhiên đến năm 1939 khi tái tổ chức lại tỉnh , tỉnh lị được chuyển đến An Đông ( Đan Đông ngày nay ) , một đô thị biên giới chiến lược giữa Mãn Châu Quốc và Triều Tiên , và nằm trên tuyến đường sắt từ Triều Tiên đến Thẩm Dương .",An Đông (tỉnh)
40086,add_21979,False,tỉnh nào đông dân nhất Trung Quốc,"Vương Hiểu Đông là người Hán sinh tháng 1 năm 1960 ở Tín Phong , tỉnh Giang Tây .",Vương Hiểu Đông
40087,add_21980,False,tỉnh nào đông dân nhất Trung Quốc,"Tháng 9 năm 2000 đến tháng 7 năm 2003 , ông theo học chuyên ngành chính trị học lớp nghiên cứu sinh tại chức cán bộ cấp tỉnh bộ tại Trường Đảng Trung ương Đảng Cộng sản Trung Quốc .",Vương Hiểu Đông


In [23]:
merge_df.label.value_counts()

False    17248
False    12370
True     10480
Name: label, dtype: int64

In [26]:
label_set = set()
for index, row in merge_df.iterrows():
    label = row['label']
    label_set.add(label)

In [27]:
label_set

{False, 'False', True}